# How to Manage Attribute and Data Access in Python Classes
## Emposing restrictions to protect your code
![](images/pexels.jpg)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@pixabay?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pixabay</a>
        on 
        <a href='https://www.pexels.com/photo/black-android-smartphone-on-top-of-white-book-39584/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels</a>
    </strong>
</figcaption>

In our open-source Python community, all class, OOP data is technically public. If you want, you can change much of the internal logic by tapping into the source code and ruin everything for yourself. If your programming background is from another language such as Java or C, you might be surprised that is the case. 
You might be even more surprised that these type of *screw-ups* almost never happen among Python developers. So, what is our secret?

Turns out, the fundamental principle behind much of Python code design is "we are all adults here". If you have never heard this before, now is the time you learned that we, as Python developers should trust and respect each other's code.

All OOP code in Python have universal naming conventions and special keywords (a *code* among us, if you will) that indicate the fact that even though the code is public, it is not meant for external use. These would be the parts of the code that work soundless under the hood and never meant for public use.

In this article, you will learn about these naming conventions and special attributes called properties that help you manage access to the private internals of your Python classes.
